# Classificação e Regressão com Redes Neurais

In [1]:
# Adaptação para utilizar a sintaxe do TensorFlor 1.x no TensorFlow 2.x
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error

<br/> 
## 1. Perceptron de 1 Camada

In [3]:
# Define dados de entrada (Operador AND)
x = np.array([[0.,0.],[0.,1.],[1.,0.],[1.,1.]])
y = np.array([[0.], [0.], [0.], [1.]])

In [4]:
# Inicializa os pesos
w = tf.Variable(tf.zeros([2,1], dtype=tf.float64))

# Define o calculo da camada de saída
output_layer = tf.matmul(x, w)

# Define a função de ativação (step function)
output_layer_result = tf.cast(tf.to_float(tf.math.greater_equal(output_layer, 1)), tf.float64)
 # greater_equal: retorna True/False
 # to_float: transforma em float
 # cast: transforma em um formato especifico

# Define o calculo do Erro
erro = tf.subtract(y, output_layer_result)

Instructions for updating:
Use `tf.cast` instead.


In [5]:
# peno_n+1 = peso_n + (learn_rate * entradas * erro)
delta = tf.matmul(x, erro, transpose_a = True)

compute_new_w = tf.assign(w, tf.add(w, tf.multiply(delta, 0.1)))
 # assign: atualiza o valor de w

In [6]:
var_init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(var_init)
    #print(sess.run(output_layer), '\n\nPós Func Ativação:')
    #print(sess.run(output_layer_result), '\n\nErro:')
    #print(sess.run(erro), '\n\nNovo Peso:')
    #print(sess.run(compute_new_w), '\n')
    
    for i in range(15):
        erro_total, _ = sess.run((erro, compute_new_w))
         # erro_total é vetor com o erro de cada um dos registros
        erro_soma = tf.reduce_sum(erro_total) # calcula o somatorio do erro de todos os registros
        print('Epoch {}: Erro = {}'.format(i, sess.run(erro_soma)))
        
        if erro_soma.eval() == 0:
            break
    
    w_final = sess.run(w)

Epoch 0: Erro = 1.0
Epoch 1: Erro = 1.0
Epoch 2: Erro = 1.0
Epoch 3: Erro = 1.0
Epoch 4: Erro = 1.0
Epoch 5: Erro = 0.0


In [7]:
w_final

array([[0.5],
       [0.5]])

In [8]:
# Realiza a previsão
output_layer_predict = tf.matmul(x, w_final)
output_layer_result_predict = tf.cast(tf.to_float(tf.math.greater_equal(output_layer_predict, 1)), tf.float64)

with tf.Session() as sess:
    sess.run(var_init)
    print(sess.run(output_layer_result_predict))

[[0.]
 [0.]
 [0.]
 [1.]]


<br/><br/> 
## 2. Classificação

### 2.1 Classificação Binária (Operador XOR)

In [9]:
# Define dados de entrada (Operador XOR)
x = np.array([[0.,0.],[0.,1.],[1.,0.],[1.,1.]], dtype=np.float32)
y = np.array([[0.], [1.], [1.], [0.]])

In [10]:
# Define estrutura da RNN
input_neurons = 2
hidden_neurons = 3
output_neurons = 1

In [11]:
# Define os placeholders de entrada e saída
x_ph = tf.placeholder(dtype=tf.float32, shape=(4, input_neurons), name='x_placeholder')
y_ph = tf.placeholder(dtype=tf.float32, shape=(4, output_neurons), name='y_placeholder')

# Inicializa os pesos e bias
w = {'hidden': tf.Variable(tf.random_normal([input_neurons, hidden_neurons]), name='w_hidden'),
     'output': tf.Variable(tf.random_normal([hidden_neurons, output_neurons]), name='w_output')}

bias = {'hidden': tf.Variable(tf.random_normal([hidden_neurons]), name='bias_hidden'),
        'output': tf.Variable(tf.random_normal([output_neurons]), name='bias_output')}

In [12]:
# Processo de FeedForward
hidden_layer = tf.add(tf.matmul(x_ph, w['hidden']), bias['hidden'])
hidden_layer_result = tf.sigmoid(hidden_layer)

output_layer = tf.add(tf.matmul(hidden_layer_result, w['output']), bias['output'])
output_layer_result = tf.sigmoid(output_layer)

In [13]:
# Calculo do Erro
erro = tf.losses.mean_squared_error(y_ph, output_layer_result)

# Processo de BackPropagation
otimizador = tf.train.GradientDescentOptimizer(learning_rate=0.3).minimize(erro)

In [14]:
var_init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(var_init)
    for epoch in range(10000):
        erro_medio = 0
        _, loss = sess.run([otimizador, erro], feed_dict={x_ph: x, y_ph: y})
        
        if epoch % 200 == 0:
            erro_medio += loss / 4
            print('Epoch {}: {}'.format(epoch, erro_medio))
    
    w_final, bias_final = sess.run([w, bias])

Epoch 0: 0.08340046554803848
Epoch 200: 0.05140066146850586
Epoch 400: 0.046587347984313965
Epoch 600: 0.04288885369896889
Epoch 800: 0.04011023789644241
Epoch 1000: 0.03799761086702347
Epoch 1200: 0.03621741384267807
Epoch 1400: 0.03416530042886734
Epoch 1600: 0.03015596978366375
Epoch 1800: 0.02274371311068535
Epoch 2000: 0.015623115003108978
Epoch 2200: 0.010914630256593227
Epoch 2400: 0.007959224283695221
Epoch 2600: 0.006046987138688564
Epoch 2800: 0.004760916344821453
Epoch 3000: 0.0038623830769211054
Epoch 3200: 0.00321214203722775
Epoch 3400: 0.002726680366322398
Epoch 3600: 0.0023542630951851606
Epoch 3800: 0.002061778912320733
Epoch 4000: 0.0018273736350238323
Epoch 4200: 0.0016361856833100319
Epoch 4400: 0.0014778526965528727
Epoch 4600: 0.001344967051409185
Epoch 4800: 0.0012321219546720386
Epoch 5000: 0.001135295140556991
Epoch 5200: 0.001051444560289383
Epoch 5400: 0.0009782301494851708
Epoch 5600: 0.00091382279060781
Epoch 5800: 0.0008567879558540881
Epoch 6000: 0.000805

In [15]:
w_final

{'hidden': array([[-6.3256917, -2.48976  , -6.0571938],
        [-5.522573 ,  5.8612833,  3.3859425]], dtype=float32),
 'output': array([[-7.111556 ],
        [-7.116505 ],
        [ 7.5502524]], dtype=float32)}

In [16]:
bias_final

{'hidden': array([ 1.7153709 , -0.00413651, -1.0670974 ], dtype=float32),
 'output': array([3.7018795], dtype=float32)}

In [17]:
# Predict
hidden_layer_predict = tf.add(tf.matmul(x, w_final['hidden']), bias_final['hidden'])
hidden_layer_result_predict = tf.sigmoid(hidden_layer_predict)

output_layer_predict = tf.add(tf.matmul(hidden_layer_result_predict, w_final['output']), bias_final['output'])
output_layer_result_predict = tf.sigmoid(output_layer_predict)

with tf.Session() as sess:
    sess.run(var_init)
    previsoes = sess.run(output_layer_result_predict)

previsoes

array([[0.01900709],
       [0.96527964],
       [0.9566862 ],
       [0.04732471]], dtype=float32)

<br/> 
### 2.2 Classificação Multiclasse (Base Iris)

In [18]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder

In [19]:
iris = load_iris()

In [20]:
x = iris.data
y = iris.target

In [21]:
# Pre processamento
scaler_x = StandardScaler()
x = scaler_x.fit_transform(x)

encoder = OneHotEncoder(categorical_features=[0])
y = encoder.fit_transform(y.reshape(-1, 1)).toarray()

/Users/brunomaciel/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/brunomaciel/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [23]:
# Estrutura da RNN
input_neurons = x.shape[1]
hidden_neurons = 4
output_neurons = y.shape[1]

In [24]:
# Define os placeholders de entrada e saída
x_ph = tf.placeholder(dtype=tf.float32, shape=(None, input_neurons), name='x_placeholder')
y_ph = tf.placeholder(dtype=tf.float32, shape=(None, output_neurons), name='y_placeholder')

# Inicializa os pesos e bias
w = {'hidden': tf.Variable(tf.random_uniform(shape=[input_neurons, hidden_neurons]), name='w_hidden'),
     'output': tf.Variable(tf.random_uniform(shape=[hidden_neurons, output_neurons]), name='w_output')}

bias = {'hidden': tf.Variable(tf.random_uniform(shape=[hidden_neurons]), name='bias_hidden'),
        'output': tf.Variable(tf.random_uniform(shape=[output_neurons]), name='bias_output')}

In [25]:
def mlp(x, w, bias):
    hidden_layer = tf.add(tf.matmul(x, w['hidden']), bias['hidden'])
    hidden_layer_result = tf.nn.relu(hidden_layer)

    output_layer = tf.add(tf.matmul(hidden_layer_result, w['output']), bias['output'])
    
    return output_layer

In [26]:
# Processo de FeedForward
output_layer = mlp(x_ph, w, bias)

# Calculo do Erro
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output_layer, labels=y_ph))

# Processo de BackPropagation
otimizador = tf.train.AdamOptimizer(learning_rate=0.001).minimize(erro)

In [27]:
# Define parametros do treinamento
BATCH_SIZE = 8
EPOCHS = 3000

In [28]:
var_init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(var_init)
    for epoch in range(EPOCHS):
        erro_medio = 0
        n_batches = int(len(x_train) / BATCH_SIZE)
        x_inBatches = np.array_split(x_train, n_batches) # Divide dados de treinamentos em batches
        y_inBatches = np.array_split(y_train, n_batches)
        
        for i in range(n_batches):
            _, loss = sess.run([otimizador, erro], feed_dict={x_ph: x_inBatches[i], y_ph: y_inBatches[i]})
            erro_medio += loss / n_batches
            
        if epoch % 500 == 0:
            print('Epoch {}: {}'.format(epoch+1, erro_medio))
    
    w_final, bias_final = sess.run([w, bias])

Epoch 1: 2.2402146962972784
Epoch 501: 0.04957659456592339
Epoch 1001: 0.03702863845795106
Epoch 1501: 0.035877410142207876
Epoch 2001: 0.03546482137776016
Epoch 2501: 0.03525346525622388


In [29]:
# Predict
previsoes = mlp(x_ph, w_final, bias_final)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(previsoes, feed_dict={x_ph: x_test})
    r2 = sess.run(tf.nn.softmax(r1)) # resposta em probabilidade
    r3 = sess.run(tf.argmax(r2, axis=1)) # resposta final

In [30]:
y_test_1d = np.argmax(y_test, axis=1)

In [31]:
score = accuracy_score(y_test_1d, r3)
score

0.9333333333333333

<br/> 
### 2.3 Classificação Binária com Estimators (Base Censo) 

In [32]:
data = pd.read_csv('census.csv')
data.shape

(32561, 15)

In [33]:
data.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [34]:
# Divisão entre atributos previsores e target
x = data.drop('income', axis=1)
y = data['income']

In [35]:
# Preprocessamento nas colunas

# Label encoder da variavel target
y = y.map({' >50K': 1, ' <=50K' : 0})

# Atributos categoricos
workclass_col = tf.feature_column.categorical_column_with_hash_bucket(key='workclass', hash_bucket_size=100)
education_col = tf.feature_column.categorical_column_with_hash_bucket(key='education', hash_bucket_size=100)
marital_col = tf.feature_column.categorical_column_with_hash_bucket(key='marital-status', hash_bucket_size=100)
occupation_col = tf.feature_column.categorical_column_with_hash_bucket(key='occupation', hash_bucket_size=100)
relationship_col = tf.feature_column.categorical_column_with_hash_bucket(key='relationship', hash_bucket_size=100)
race_col = tf.feature_column.categorical_column_with_hash_bucket(key='race', hash_bucket_size=100)
country_col = tf.feature_column.categorical_column_with_hash_bucket(key='native-country', hash_bucket_size=100)
sex_col = tf.feature_column.categorical_column_with_vocabulary_list(key='sex', vocabulary_list=[' Male', ' Female'])

embedded_workclass = tf.feature_column.embedding_column(workclass_col, dimension=data['workclass'].unique().shape[0])
embedded_education = tf.feature_column.embedding_column(education_col, dimension=data['education'].unique().shape[0])
embedded_marital = tf.feature_column.embedding_column(marital_col, dimension=data['marital-status'].unique().shape[0])
embedded_occupation = tf.feature_column.embedding_column(occupation_col, dimension=data['occupation'].unique().shape[0])
embedded_relationship = tf.feature_column.embedding_column(relationship_col, dimension=data['relationship'].unique().shape[0])
embedded_race = tf.feature_column.embedding_column(race_col, dimension=data['race'].unique().shape[0])
embedded_country = tf.feature_column.embedding_column(country_col, dimension=data['native-country'].unique().shape[0])
embedded_sex = tf.feature_column.embedding_column(sex_col, dimension=data['sex'].unique().shape[0])

In [36]:
# Atributos numéricos
age_col = tf.feature_column.numeric_column(key='age')
weight_col = tf.feature_column.numeric_column(key='final-weight')
educationNum_col = tf.feature_column.numeric_column(key='education-num')
capGain_col = tf.feature_column.numeric_column(key='capital-gain')
capLoos_col = tf.feature_column.numeric_column(key='capital-loos')
hour_col = tf.feature_column.numeric_column(key='hour-per-week')

In [37]:
colunas = [age_col, embedded_workclass, weight_col, embedded_education, educationNum_col, embedded_marital,
           embedded_occupation, embedded_relationship, embedded_race, embedded_sex, capGain_col, capLoos_col,
           hour_col, embedded_country]

In [38]:
# Divisão entre dados treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [39]:
funcao_train = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train,
                                                  batch_size=32, num_epochs=None, shuffle=True)
funcao_test = tf.estimator.inputs.pandas_input_fn(x=x_test, y=y_test, 
                                                  batch_size=32, num_epochs=1, shuffle=False)

In [ ]:
classificador = tf.estimator.DNNClassifier(hidden_units=[8,8], feature_columns=colunas, n_classes=2)
classificador.train(input_fn=funcao_train, steps=10000)

In [41]:
classificador.evaluate(input_fn=funcao_test)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
The value of AUC returned by this may race with the update so this is deprected. Please use tf.keras.metrics.AUC instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-07T14:58:50Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/ps/3gh7bfwd77b_xjwn226rnrqw0000gn/T/tmpl79pg6ay/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 3.12126s
INFO:tensorflow:Finished evaluation at 2020-04-07-14:58:53
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.77152216, accuracy_baseline = 0.7547344, auc = 0.5519725, auc_precision_recall = 0.5574247, average_loss = 9.238142, global_step = 10000, label/mean = 0.24526563, loss = 294.92618, precision = 0.8761468, prediction/mean = 0.022699889, 

{'accuracy': 0.77152216,
 'accuracy_baseline': 0.7547344,
 'auc': 0.5519725,
 'auc_precision_recall': 0.5574247,
 'average_loss': 9.238142,
 'label/mean': 0.24526563,
 'loss': 294.92618,
 'precision': 0.8761468,
 'prediction/mean': 0.022699889,
 'recall': 0.07971619,
 'global_step': 10000}

<br/> 
### 2.3.1 Padronização

Normalização: 
 - $ x_{norm} = \frac{ x - min(x)} { max(x) - min(x)} $
 - $ 0 \leq x_{norm} \leq 1 $

Padronização: 
 - $ x_{std} = \frac{ x - mean(x)} { std(x)} $
 - permite valores negativos


In [42]:
AGE_AVG, AGE_STDDEV = data.age.mean(), data.age.std()
weight_AVG, weight_STDDEV = data.age.mean(), data['final-weight'].std()
educationNum_AVG, educationNum_STDDEV = data.age.mean(), data['education-num'].std()
capGain_AVG, capGain_STDDEV = data.age.mean(), data['capital-gain'].std()
CAPLOOS_AVG, CAPLOOS_STDDEV = data.age.mean(), data['capital-loos'].std()
HOUR_AVG, HOUR_STDDEV = data.age.mean(), data['hour-per-week'].std()

def padroniza_age(valor):
    # x = (x - mean(x)) / std_dev(x)
    numerador = tf.subtract(tf.cast(valor, tf.float32), tf.constant(AGE_AVG))
    return tf.divide(numerador, tf.constant(AGE_STDDEV))

def padroniza_weight(valor):
    numerador = tf.subtract(tf.cast(valor, tf.float32), tf.constant(weight_AVG))
    return tf.divide(numerador, tf.constant(weight_STDDEV))

def padroniza_education(valor):
    numerador = tf.subtract(tf.cast(valor, tf.float32), tf.constant(educationNum_AVG))
    return tf.divide(numerador, tf.constant(educationNum_STDDEV))

def padroniza_capGain(valor):
    numerador = tf.subtract(tf.cast(valor, tf.float32), tf.constant(capGain_AVG))
    return tf.divide(numerador, tf.constant(capGain_STDDEV))

def padroniza_capLoos(valor):
    numerador = tf.subtract(tf.cast(valor, tf.float32), tf.constant(CAPLOOS_AVG))
    return tf.divide(numerador, tf.constant(CAPLOOS_STDDEV))

def padroniza_hour(valor):
    numerador = tf.subtract(tf.cast(valor, tf.float32), tf.constant(HOUR_AVG))
    return tf.divide(numerador, tf.constant(HOUR_STDDEV))

In [43]:
# Atributos numéricos
age_col = tf.feature_column.numeric_column(key='age', normalizer_fn=padroniza_age)
weight_col = tf.feature_column.numeric_column(key='final-weight', normalizer_fn=padroniza_weight)
educationNum_col = tf.feature_column.numeric_column(key='education-num', normalizer_fn=padroniza_education)
capGain_col = tf.feature_column.numeric_column(key='capital-gain', normalizer_fn=padroniza_capGain)
capLoos_col = tf.feature_column.numeric_column(key='capital-loos', normalizer_fn=padroniza_capLoos)
hour_col = tf.feature_column.numeric_column(key='hour-per-week', normalizer_fn=padroniza_hour)

<br/><br/> 
## 3. Regressão com Estimators

In [44]:
data = pd.read_csv('house_prices.csv')
data.shape

(21613, 21)

In [45]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [46]:
data.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [47]:
use_columns = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 
           'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 
           'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long']
data = pd.read_csv('house_prices.csv', usecols=use_columns)
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045


In [48]:
x = data.drop('price', axis=1)
y = data['price']

In [49]:
# Pre processamento dos dados
scaler_x = MinMaxScaler()
x_scaled = scaler_x.fit_transform(x)
x_scaled = pd.DataFrame(x_scaled, columns=x.columns)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))
y_scaled = pd.Series(y_scaled.reshape(-1,), name='price')

In [50]:
# Cria as colunas de atributos previsores
colunas = [tf.feature_column.numeric_column(key=col) for col in use_columns[1:]]

In [51]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.3)

In [52]:
# Define as funções de treinamento e previsão
funcao_train = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train, batch_size=32, num_epochs=None, shuffle=True)

funcao_predict = tf.estimator.inputs.pandas_input_fn(x=x_test, shuffle=False)

In [53]:
# Cria o regressor
regressor = tf.estimator.DNNRegressor(hidden_units=(8,8,8), feature_columns=colunas)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/ps/3gh7bfwd77b_xjwn226rnrqw0000gn/T/tmp0spmftk0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Realiza o treinamento
regressor.train(input_fn=funcao_train, steps=20000)

In [55]:
previsao_gen = regressor.predict(input_fn=funcao_predict)
list(previsao_gen)[0:5]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/ps/3gh7bfwd77b_xjwn226rnrqw0000gn/T/tmp0spmftk0/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([0.02033477], dtype=float32)},
 {'predictions': array([0.02544899], dtype=float32)},
 {'predictions': array([0.04228958], dtype=float32)},
 {'predictions': array([0.03751269], dtype=float32)},
 {'predictions': array([0.03039036], dtype=float32)}]

In [56]:
# Realiza a previsão
previsoes_scaled = []
for previsao in regressor.predict(input_fn=funcao_predict):
    previsoes_scaled.append(previsao['predictions'])

previsoes_scaled = np.array(previsoes_scaled).reshape(-1, 1)
previsoes = scaler_y.inverse_transform(previsoes_scaled)
previsoes

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/ps/3gh7bfwd77b_xjwn226rnrqw0000gn/T/tmp0spmftk0/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


array([[230052.58],
       [269048.56],
       [397458.1 ],
       ...,
       [605558.25],
       [947822.7 ],
       [390693.5 ]], dtype=float32)

In [57]:
y_test_unscaled = scaler_y.inverse_transform(y_test.values.reshape(-1, 1))
erro = mean_absolute_error(y_test_unscaled, previsoes)
erro

94218.02958002391